## 1. pySpark DataFrame

In [ ]:
!pip install pyspark
!curl -L -o "/usr/local/lib/python3.6/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar" http://dl.bintray.com/spark-packages/maven/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar
!apt-get install openjdk-8-jdk-headless -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 417, in run
    _, build_failures = build(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/wheel_builder.py", line 320, in build
    wheel_file = _build_one(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/wheel_builder.py", line 194, in _build_one
    wheel_path = _build_one_inside_env(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/wheel_builder.py", line 241, in _build_one_inside_en

In [ ]:
import os
import numpy as np
import pandas as pd
import nltk
import pyspark
import pyspark.sql.functions as F
import sys
import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from datetime import datetime
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from pyspark import SparkContext, SparkConf
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import col, sum as pyspark_sum

ModuleNotFoundError: ignored

In [ ]:
os.environ["JAVA_HOME"]= "/usr/lib/jvm/java-8-openjdk-amd64"

sc= pyspark.SparkContext()
spark= SparkSession.builder.getOrCreate()

In [ ]:
path= "/content/drive/MyDrive/23 U 4-2/빅데이터처리및응용/Data/"

movie_sample= pd.read_csv(path+ "1206 sample_data_pp.csv")
movie_sample.head()

In [ ]:
def parse_list(s):
  return [float(x) for x in s.strip('][').split(',')]

columns_to_convert = ['title', 'era', 'genres', 'tag', 'storyline', 'runtime', 'continent', 'age_rating']
for col in columns_to_convert:
  movie_sample[col] = movie_sample[col].apply(parse_list)

movie_sample.head()

In [ ]:
schema= [
    StructField("movieId", StringType(), False),
    StructField("title", ArrayType(FloatType()), False),
    StructField("genres", ArrayType(FloatType()), True),
    StructField("storyline", ArrayType(FloatType()), True),
    StructField("country", ArrayType(FloatType()), True),
    StructField("release_year", ArrayType(FloatType()), True),
    StructField("runtime", ArrayType(FloatType()), True),
    StructField("age_rating", ArrayType(FloatType()), True),
    StructField("tag", ArrayType(FloatType()), True)
]

spMovies= spark.createDataFrame(movie_sample, schema= StructType(schema))
spMovies.show()

## 2. pandas DataFrame

In [3]:
import os
import numpy as np
import pandas as pd
import nltk
import sys
import warnings
warnings.filterwarnings('ignore')

from scipy.spatial.distance import cosine
from collections import Counter
from datetime import datetime
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
path= "/content/drive/MyDrive/23 U 4-2/빅데이터처리및응용/Data/"

movie_sample= pd.read_csv(path+ "1210 all_pp.csv")
movie_sample.head()

,movieId,title,era,genres,tag,storyline,runtime,continent,age_rating
0,1,"[-0.014765826907629767,-0.0001763248195250829,...","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, ...","[-0.06418165498180316,-0.009971046587452292,-0...","[0.019509797799400985,-0.022897314243508795,-0...",[-0.8024960503192684],"[0, 0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]"
1,2,"[-0.020792629569768906,-0.005490744020789862,-...","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[-0.025203625322319567,-0.023397489140431084,0...","[0.015569244019747148,-0.0014084530279929327,0...",[-0.04949848793283062],"[0, 0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]"
2,3,"[-0.0010529104038141668,-0.003023075812961906,...","[0, 0, 0, 1, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.02361708382765452,-0.006075011993137498,0....","[0.0034510755504015833,-0.013722939333092654,-...",[-0.14771556128758348],"[0, 0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]"
3,4,"[-0.003295399248600006,-0.007947875885292888,-...","[0, 0, 0, 1, 0]","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.008344948175363243,-0.024232767755165696,0...","[0.005609218422950317,-0.022722859384838298,0....",[0.6052820010988548],"[0, 0, 0, 1, 0, 0]","[0, 0, 0, 0, 0]"
4,5,"[0.007662885689309664,-0.004842474711559979,-0...","[0, 0, 0, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.013301791623234749,-0.008611441900332768,0...","[0.004049535797726687,-0.010654402755038893,-0...",[0.015979560970337654],"[0, 0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]"


## 3. pandas Cosine-Similarity

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
import ast

### 3.1 Cosine - All

In [4]:
movie_sample= pd.read_csv(path+ "1210 all_pp.csv")

In [5]:
# Cosine Similarity
feature_columns = ['title', 'era', 'genres', 'tag', 'storyline', 'runtime', 'continent', 'age_rating']

def list_to_array(lst):
    return np.array(ast.literal_eval(lst)) if isinstance(lst, str) else np.array(lst)

for col in feature_columns:
    movie_sample[col] = movie_sample[col].apply(list_to_array)

cosine_similarities = {}
for col in feature_columns:
    cosine_sim = cosine_similarity(list(movie_sample[col]))
    cosine_similarities[col] = cosine_sim

movie_ids = movie_sample['movieId'].tolist()
cosine_similarity_df = pd.DataFrame(index=movie_ids, columns=movie_ids)

for i in range(len(movie_ids)):
    for j in range(len(movie_ids)):
        similarities = []
        for col in feature_columns:
            similarities.append(cosine_similarities[col][i][j])
        cosine_similarity_df.at[movie_ids[i], movie_ids[j]] = sum(similarities) / len(similarities)

cosine_similarity_df = cosine_similarity_df.astype(float)

In [6]:
# MinMax-Scaling
scaler = MinMaxScaler()
scaled_all = cosine_similarity_df.copy()
scale_col= list(cosine_similarity_df.columns)

scaled_all[scale_col] = scaler.fit_transform(scaled_all[scale_col])
scaled_all.head()

,1,2,3,4,5,6,7,8,9,10,...,191005,193565,193567,193571,193573,193581,193583,193585,193587,193609
1,1.000000,0.876327,0.716804,0.457971,0.617727,0.508218,0.395936,0.849832,0.447652,0.434804,...,0.323645,0.545659,0.465674,0.377197,0.548296,0.508439,0.266175,0.616943,0.506044,0.865413
2,0.875729,1.000000,0.665633,0.475699,0.569404,0.553243,0.416010,0.889508,0.543152,0.444473,...,0.311305,0.410299,0.386816,0.305902,0.479891,0.448355,0.223821,0.636172,0.454245,0.743381
3,0.718801,0.669587,1.000000,0.630407,0.575003,0.569360,0.525845,0.628621,0.443755,0.446823,...,0.298588,0.500797,0.452085,0.348682,0.422717,0.612429,0.260799,0.675633,0.519121,0.855869
4,0.427382,0.445211,0.571714,1.000000,0.762320,0.806226,0.612316,0.391407,0.698074,0.528154,...,0.483952,0.181812,0.249821,0.554674,0.222749,0.170516,0.466419,0.512938,0.178643,0.526398
5,0.635450,0.591340,0.591828,0.862821,1.000000,0.820115,0.646798,0.573423,0.730685,0.557048,...,0.546175,0.286394,0.252738,0.584853,0.353173,0.272550,0.535068,0.424955,0.236189,0.615567


In [7]:
scaled_all.to_csv(path+ '1210 CosSim_All.csv', encoding= 'utf-8', index= False)

### 3.2 Cosine - Title

In [8]:
movie_sample= pd.read_csv(path+ "1210 all_pp.csv")
movie_sample= movie_sample[['movieId', 'title']]
movie_sample.head()

,movieId,title
0,1,"[-0.014765826907629767,-0.0001763248195250829,..."
1,2,"[-0.020792629569768906,-0.005490744020789862,-..."
2,3,"[-0.0010529104038141668,-0.003023075812961906,..."
3,4,"[-0.003295399248600006,-0.007947875885292888,-..."
4,5,"[0.007662885689309664,-0.004842474711559979,-0..."


In [9]:
# Cosine-Similarity
feature_columns = ['title']

def list_to_array(lst):
    return np.array(ast.literal_eval(lst)) if isinstance(lst, str) else np.array(lst)

for col in feature_columns:
    movie_sample[col] = movie_sample[col].apply(list_to_array)

cosine_similarities = {}
for col in feature_columns:
    cosine_sim = cosine_similarity(list(movie_sample[col]))
    cosine_similarities[col] = cosine_sim

movie_ids = movie_sample['movieId'].tolist()
cosine_similarity_df = pd.DataFrame(index=movie_ids, columns=movie_ids)

for i in range(len(movie_ids)):
    for j in range(len(movie_ids)):
        similarities = []
        for col in feature_columns:
            similarities.append(cosine_similarities[col][i][j])
        cosine_similarity_df.at[movie_ids[i], movie_ids[j]] = sum(similarities) / len(similarities)

cosine_similarity_df = cosine_similarity_df.astype(float)

In [10]:
# MinMax-scaling
scaler = MinMaxScaler()
scaled_title = cosine_similarity_df.copy()
scale_col= list(cosine_similarity_df.columns)

scaled_title[scale_col] = scaler.fit_transform(scaled_title[scale_col])
scaled_title.head()

,1,2,3,4,5,6,7,8,9,10,...,191005,193565,193567,193571,193573,193581,193583,193585,193587,193609
1,1.000000,0.951071,0.977894,0.971615,0.927924,0.979308,0.951071,0.870856,0.987060,0.951071,...,0.984479,0.623670,0.648939,0.945481,0.644483,0.933212,0.899122,0.984479,0.915482,0.985180
2,0.938674,1.000000,0.870291,0.891703,0.785248,0.984918,1.000000,0.729533,0.940772,1.000000,...,0.980496,0.406150,0.430658,0.992908,0.425919,0.791020,0.718380,0.980496,0.750132,0.979183
3,0.977385,0.894127,1.000000,0.980200,0.971211,0.942383,0.894127,0.901100,0.972213,0.894127,...,0.940223,0.687530,0.723143,0.891331,0.720256,0.971301,0.948670,0.940223,0.959031,0.946727
4,0.971829,0.914247,0.980792,1.000000,0.950589,0.954871,0.914247,0.922885,0.957110,0.914247,...,0.950687,0.639354,0.676674,0.907096,0.672976,0.947123,0.915620,0.950687,0.940161,0.961122
5,0.922334,0.815372,0.969677,0.946352,1.000000,0.875499,0.815372,0.932568,0.906534,0.815372,...,0.875384,0.747093,0.785092,0.812620,0.785438,0.997935,0.968967,0.875384,0.979845,0.885725


In [11]:
scaled_title.to_csv(path+ '1210 CosSim_Title.csv', encoding= 'utf-8', index= False)

### 3.3 Cosine - Genres

In [12]:
movie_sample= pd.read_csv(path+ "1210 all_pp.csv")
movie_sample= movie_sample[['movieId', 'genres']]
movie_sample.head()

,movieId,genres
0,1,"[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, ..."
1,2,"[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
2,3,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,4,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [13]:
# Cosine-Similarity
feature_columns = ['genres']

def list_to_array(lst):
    return np.array(ast.literal_eval(lst)) if isinstance(lst, str) else np.array(lst)

for col in feature_columns:
    movie_sample[col] = movie_sample[col].apply(list_to_array)

cosine_similarities = {}
for col in feature_columns:
    cosine_sim = cosine_similarity(list(movie_sample[col]))
    cosine_similarities[col] = cosine_sim

movie_ids = movie_sample['movieId'].tolist()
cosine_similarity_df = pd.DataFrame(index=movie_ids, columns=movie_ids)

for i in range(len(movie_ids)):
    for j in range(len(movie_ids)):
        similarities = []
        for col in feature_columns:
            similarities.append(cosine_similarities[col][i][j])
        cosine_similarity_df.at[movie_ids[i], movie_ids[j]] = sum(similarities) / len(similarities)

cosine_similarity_df = cosine_similarity_df.astype(float)

In [14]:
# MinMax-scaling
scaler = MinMaxScaler()
scaled_genres = cosine_similarity_df.copy()
scale_col= list(cosine_similarity_df.columns)

scaled_genres[scale_col] = scaler.fit_transform(scaled_genres[scale_col])
scaled_genres.head()

,1,2,3,4,5,6,7,8,9,10,...,191005,193565,193567,193571,193573,193581,193583,193585,193587,193609
1,1.000000,0.774597,0.316228,0.258199,0.447214,0.0,0.316228,0.632456,0.0,0.258199,...,0.447214,0.447214,0.316228,0.316228,0.447214,0.670820,0.774597,0.00000,0.316228,0.447214
2,0.774597,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.816497,0.0,0.333333,...,0.288675,0.000000,0.000000,0.000000,0.000000,0.288675,0.333333,0.00000,0.000000,0.000000
3,0.316228,0.000000,1.000000,0.816497,0.707107,0.0,1.000000,0.000000,0.0,0.000000,...,0.353553,0.353553,0.000000,0.500000,0.000000,0.353553,0.408248,0.00000,0.000000,0.707107
4,0.258199,0.000000,0.816497,1.000000,0.577350,0.0,0.816497,0.000000,0.0,0.000000,...,0.288675,0.288675,0.408248,0.816497,0.000000,0.288675,0.333333,0.57735,0.000000,0.577350
5,0.447214,0.000000,0.707107,0.577350,1.000000,0.0,0.707107,0.000000,0.0,0.000000,...,0.500000,0.500000,0.000000,0.707107,0.000000,0.500000,0.577350,0.00000,0.000000,1.000000


In [15]:
scaled_genres.to_csv(path+ '1210 CosSim_Genres.csv', encoding= 'utf-8', index= False)

### 3.4 Cosine - Tag

In [16]:
movie_sample= pd.read_csv(path+ "1210 all_pp.csv")
movie_sample= movie_sample[['movieId', 'tag']]
movie_sample.head()

,movieId,tag
0,1,"[-0.06418165498180316,-0.009971046587452292,-0..."
1,2,"[-0.025203625322319567,-0.023397489140431084,0..."
2,3,"[-0.02361708382765452,-0.006075011993137498,0...."
3,4,"[-0.008344948175363243,-0.024232767755165696,0..."
4,5,"[-0.013301791623234749,-0.008611441900332768,0..."


In [17]:
# Cosine-Similarity
feature_columns = ['tag']

def list_to_array(lst):
    return np.array(ast.literal_eval(lst)) if isinstance(lst, str) else np.array(lst)

for col in feature_columns:
    movie_sample[col] = movie_sample[col].apply(list_to_array)

cosine_similarities = {}
for col in feature_columns:
    cosine_sim = cosine_similarity(list(movie_sample[col]))
    cosine_similarities[col] = cosine_sim

movie_ids = movie_sample['movieId'].tolist()
cosine_similarity_df = pd.DataFrame(index=movie_ids, columns=movie_ids)

for i in range(len(movie_ids)):
    for j in range(len(movie_ids)):
        similarities = []
        for col in feature_columns:
            similarities.append(cosine_similarities[col][i][j])
        cosine_similarity_df.at[movie_ids[i], movie_ids[j]] = sum(similarities) / len(similarities)

cosine_similarity_df = cosine_similarity_df.astype(float)

In [18]:
# MinMax-scaling
scaler = MinMaxScaler()
scaled_tag = cosine_similarity_df.copy()
scale_col= list(cosine_similarity_df.columns)

scaled_tag[scale_col] = scaler.fit_transform(scaled_tag[scale_col])
scaled_tag.head()

,1,2,3,4,5,6,7,8,9,10,...,191005,193565,193567,193571,193573,193581,193583,193585,193587,193609
1,1.000000,0.687218,0.614179,0.409190,0.681596,0.629371,0.537308,0.664438,0.419308,0.861715,...,0.659038,0.760283,0.483130,0.778537,0.650291,0.659343,0.606805,0.525788,0.619500,0.779297
2,0.684037,1.000000,0.706921,0.576255,0.872924,0.808943,0.789704,0.881164,0.807655,0.724448,...,0.650852,0.641836,0.641033,0.671715,0.672163,0.687790,0.642853,0.615521,0.648023,0.551179
3,0.608650,0.705714,1.000000,0.843258,0.897268,0.934991,0.352193,0.497484,0.380399,0.639116,...,0.723303,0.686945,0.550524,0.603068,0.733009,0.751501,0.728151,0.916815,0.770947,0.607896
4,0.377158,0.557778,0.837094,1.000000,0.822713,0.847958,0.367642,0.450982,0.398468,0.400255,...,0.369217,0.354776,0.710817,0.398667,0.728707,0.396682,0.380482,0.935591,0.426258,0.372420
5,0.663100,0.866896,0.892835,0.822061,1.000000,0.960279,0.614962,0.722575,0.608292,0.662236,...,0.629956,0.566851,0.725368,0.589524,0.820765,0.691416,0.645235,0.856021,0.674202,0.515470


In [20]:
scaled_tag.to_csv(path+ '1210 CosSim_Tag.csv', encoding= 'utf-8', index= False)

### 3.5 Cosine - Storyline

In [21]:
movie_sample= pd.read_csv(path+ "1210 all_pp.csv")
movie_sample= movie_sample[['movieId', 'storyline']]
movie_sample.head()

,movieId,storyline
0,1,"[0.019509797799400985,-0.022897314243508795,-0..."
1,2,"[0.015569244019747148,-0.0014084530279929327,0..."
2,3,"[0.0034510755504015833,-0.013722939333092654,-..."
3,4,"[0.005609218422950317,-0.022722859384838298,0...."
4,5,"[0.004049535797726687,-0.010654402755038893,-0..."


In [25]:
# Cosine-Similarity
feature_columns = ['storyline']

def list_to_array(lst):
    return np.array(ast.literal_eval(lst)) if isinstance(lst, str) else np.array(lst)

for col in feature_columns:
    movie_sample[col] = movie_sample[col].apply(list_to_array)

cosine_similarities = {}
for col in feature_columns:
    cosine_sim = cosine_similarity(list(movie_sample[col]))
    cosine_similarities[col] = cosine_sim

movie_ids = movie_sample['movieId'].tolist()
cosine_similarity_df = pd.DataFrame(index=movie_ids, columns=movie_ids)

for i in range(len(movie_ids)):
    for j in range(len(movie_ids)):
        similarities = []
        for col in feature_columns:
            similarities.append(cosine_similarities[col][i][j])
        cosine_similarity_df.at[movie_ids[i], movie_ids[j]] = sum(similarities) / len(similarities)

cosine_similarity_df = cosine_similarity_df.astype(float)

In [23]:
feature_columns = ['storyline']

def list_to_array(lst):
    return np.array(ast.literal_eval(lst)) if isinstance(lst, str) else np.array(lst)

for col in feature_columns:
    movie_sample[col] = movie_sample[col].apply(list_to_array)

combined_features = np.hstack([movie_sample[col].tolist() for col in feature_columns])
cosine_similarities = cosine_similarity(combined_features)

movie_ids = movie_sample['movieId'].tolist()
cosine_similarity_df = pd.DataFrame(cosine_similarities, index=movie_ids, columns=movie_ids)

In [24]:
# MinMax-scaling
scaler = MinMaxScaler()
scaled_story = cosine_similarity_df.copy()
scale_col= list(cosine_similarity_df.columns)

scaled_story[scale_col] = scaler.fit_transform(scaled_story[scale_col])
scaled_story.head()

,1,2,3,4,5,6,7,8,9,10,...,191005,193565,193567,193571,193573,193581,193583,193585,193587,193609
1,1.000000,0.762706,0.864523,0.646884,0.926741,0.842085,0.879631,0.833762,0.590668,0.543110,...,0.562596,0.858061,0.862209,0.820657,0.531951,0.834700,0.460978,0.786438,0.635544,0.869872
2,0.750394,1.000000,0.716686,0.837686,0.797533,0.859303,0.854035,0.797726,0.750886,0.732725,...,0.642930,0.764276,0.662843,0.674101,0.676380,0.845499,0.766223,0.789448,0.729790,0.740015
3,0.851899,0.705562,1.000000,0.606816,0.831572,0.818048,0.857598,0.780808,0.641308,0.432655,...,0.387880,0.621030,0.827719,0.811244,0.507449,0.827062,0.495808,0.650621,0.718716,0.902909
4,0.679169,0.859801,0.673216,1.000000,0.766225,0.819598,0.806511,0.746147,0.673576,0.637942,...,0.633029,0.721753,0.603522,0.674974,0.724578,0.776707,0.812644,0.707377,0.703642,0.709824
5,0.928286,0.811578,0.849177,0.748124,1.000000,0.842166,0.942879,0.840438,0.650274,0.594525,...,0.569781,0.809547,0.789006,0.811334,0.578933,0.835694,0.548937,0.814338,0.653685,0.864950


In [26]:
scaled_story.to_csv(path+ '1210 CosSim_Story.csv', encoding= 'utf-8', index= False)

## 4. pandas Euclidean-Similarity

In [4]:
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
import ast

path= "/content/drive/MyDrive/23 U 4-2/빅데이터처리및응용/Data/"

### 4.1 Euclidean - All

In [5]:
movie_sample= pd.read_csv(path+ "1210 all_pp.csv")

In [6]:
# Euclidean Similarity
feature_columns = ['title', 'era', 'genres', 'tag', 'storyline', 'runtime', 'continent', 'age_rating']

def list_to_array(lst):
    return np.array(ast.literal_eval(lst)) if isinstance(lst, str) else np.array(lst)

for col in feature_columns:
    movie_sample[col] = movie_sample[col].apply(list_to_array)

euclidean_similarities = {}
for col in feature_columns:
    euclidean_similarities[col] = euclidean_distances(movie_sample[col].tolist())

movie_ids = movie_sample['movieId'].tolist()
euclidean_similarity_df = pd.DataFrame(index=movie_ids, columns=movie_ids)
for i in range(len(movie_ids)):
    for j in range(len(movie_ids)):
        similarities = []
        for col in feature_columns:
            similarities.append(euclidean_similarities[col][i][j])
        euclidean_similarity_df.at[movie_ids[i], movie_ids[j]] = sum(similarities) / len(similarities)
euclidean_similarity_df = euclidean_similarity_df.astype(float)

In [7]:
# MinMax-Scaling
scaler = MinMaxScaler()
scaled_all = euclidean_similarity_df.copy()
scale_col= list(euclidean_similarity_df.columns)

scaled_all[scale_col] = scaler.fit_transform(scaled_all[scale_col])
scaled_all.head()

,1,2,3,4,5,6,7,8,9,10,...,191005,193565,193567,193571,193573,193581,193583,193585,193587,193609
1,0.000000,0.084499,0.155155,0.179639,0.101060,0.248958,0.253085,0.085296,0.166165,0.249433,...,0.292667,0.245387,0.258902,0.266966,0.211859,0.269862,0.251657,0.182759,0.252649,0.119671
2,0.082491,0.000000,0.138769,0.151127,0.079988,0.206601,0.220364,0.052498,0.119760,0.205156,...,0.266275,0.253911,0.248342,0.242237,0.196687,0.267235,0.241674,0.182624,0.249998,0.130735
3,0.145721,0.133504,0.000000,0.105295,0.090522,0.200456,0.114173,0.125608,0.120654,0.172488,...,0.264819,0.225587,0.227508,0.213728,0.228986,0.205575,0.236247,0.169189,0.237784,0.094910
4,0.169940,0.146448,0.106059,0.000000,0.110379,0.152746,0.156422,0.147546,0.102339,0.192721,...,0.215271,0.231756,0.218557,0.175810,0.253252,0.281288,0.218669,0.149131,0.242231,0.106345
5,0.098257,0.079663,0.093709,0.113442,0.000000,0.186649,0.185604,0.075062,0.104906,0.206444,...,0.248925,0.220181,0.222844,0.194211,0.172280,0.249833,0.219889,0.162975,0.236833,0.071387


In [8]:
scaled_all.to_csv('/content/drive/MyDrive/23 U 4-2/빅데이터처리및응용/1211 팀프로젝트/Final Data/'+ '1210 EucSim_All.csv', encoding= 'utf-8', index= False)

### 4.2 Euclidean - Title

In [8]:
movie_sample= pd.read_csv(path+ "1210 all_pp.csv")
movie_sample= movie_sample[['movieId', 'title']]
movie_sample.head()

,movieId,title
0,1,"[-0.014765826907629767,-0.0001763248195250829,..."
1,2,"[-0.020792629569768906,-0.005490744020789862,-..."
2,3,"[-0.0010529104038141668,-0.003023075812961906,..."
3,4,"[-0.003295399248600006,-0.007947875885292888,-..."
4,5,"[0.007662885689309664,-0.004842474711559979,-0..."


In [9]:
# Euclidean Similarity
feature_columns = ['title']

def list_to_array(lst):
    return np.array(ast.literal_eval(lst)) if isinstance(lst, str) else np.array(lst)

for col in feature_columns:
    movie_sample[col] = movie_sample[col].apply(list_to_array)

euclidean_similarities = {}
for col in feature_columns:
    euclidean_similarities[col] = euclidean_distances(movie_sample[col].tolist())

movie_ids = movie_sample['movieId'].tolist()
euclidean_similarity_df = pd.DataFrame(index=movie_ids, columns=movie_ids)
for i in range(len(movie_ids)):
    for j in range(len(movie_ids)):
        similarities = []
        for col in feature_columns:
            similarities.append(euclidean_similarities[col][i][j])
        euclidean_similarity_df.at[movie_ids[i], movie_ids[j]] = sum(similarities) / len(similarities)
euclidean_similarity_df = euclidean_similarity_df.astype(float)

KeyboardInterrupt: ignored

In [ ]:
# MinMax-Scaling
scaler = MinMaxScaler()
scaled_title = euclidean_similarity_df.copy()
scale_col= list(euclidean_similarity_df.columns)

scaled_title[scale_col] = scaler.fit_transform(scaled_title[scale_col])
scaled_title.head()

In [ ]:
scaled_title.to_csv('/content/drive/MyDrive/23 U 4-2/빅데이터처리및응용/1211 팀프로젝트/Final Data/'+ '1210 EucSim_Title.csv', encoding= 'utf-8', index= False)

### 4.3 Euclidean - Genres

In [18]:
movie_sample= pd.read_csv(path+ "1210 all_pp.csv")
movie_sample= movie_sample[['movieId', 'genres']]
movie_sample.head()

,movieId,genres
0,1,"[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, ..."
1,2,"[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
2,3,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,4,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [19]:
# Euclidean Similarity
feature_columns = ['genres']

def list_to_array(lst):
    return np.array(ast.literal_eval(lst)) if isinstance(lst, str) else np.array(lst)

for col in feature_columns:
    movie_sample[col] = movie_sample[col].apply(list_to_array)

euclidean_similarities = {}
for col in feature_columns:
    euclidean_similarities[col] = euclidean_distances(movie_sample[col].tolist())

movie_ids = movie_sample['movieId'].tolist()
euclidean_similarity_df = pd.DataFrame(index=movie_ids, columns=movie_ids)
for i in range(len(movie_ids)):
    for j in range(len(movie_ids)):
        similarities = []
        for col in feature_columns:
            similarities.append(euclidean_similarities[col][i][j])
        euclidean_similarity_df.at[movie_ids[i], movie_ids[j]] = sum(similarities) / len(similarities)
euclidean_similarity_df = euclidean_similarity_df.astype(float)

KeyboardInterrupt: ignored

In [ ]:
# MinMax-Scaling
scaler = MinMaxScaler()
scaled_genres = euclidean_similarity_df.copy()
scale_col= list(euclidean_similarity_df.columns)

scaled_genres[scale_col] = scaler.fit_transform(scaled_genres[scale_col])
scaled_genres.head()

In [ ]:
scaled_genres.to_csv('/content/drive/MyDrive/23 U 4-2/빅데이터처리및응용/1211 팀프로젝트/Final Data/'+ '1210 EucSim_Genres.csv', encoding= 'utf-8', index= False)

### 4.4 Euclidean - Tag

In [14]:
movie_sample= pd.read_csv(path+ "1210 all_pp.csv")
movie_sample= movie_sample[['movieId', 'tag']]
movie_sample.head()

,movieId,tag
0,1,"[-0.06418165498180316,-0.009971046587452292,-0..."
1,2,"[-0.025203625322319567,-0.023397489140431084,0..."
2,3,"[-0.02361708382765452,-0.006075011993137498,0...."
3,4,"[-0.008344948175363243,-0.024232767755165696,0..."
4,5,"[-0.013301791623234749,-0.008611441900332768,0..."


In [15]:
# Euclidean Similarity
feature_columns = ['tag']

def list_to_array(lst):
    return np.array(ast.literal_eval(lst)) if isinstance(lst, str) else np.array(lst)

for col in feature_columns:
    movie_sample[col] = movie_sample[col].apply(list_to_array)

euclidean_similarities = {}
for col in feature_columns:
    euclidean_similarities[col] = euclidean_distances(movie_sample[col].tolist())

movie_ids = movie_sample['movieId'].tolist()
euclidean_similarity_df = pd.DataFrame(index=movie_ids, columns=movie_ids)
for i in range(len(movie_ids)):
    for j in range(len(movie_ids)):
        similarities = []
        for col in feature_columns:
            similarities.append(euclidean_similarities[col][i][j])
        euclidean_similarity_df.at[movie_ids[i], movie_ids[j]] = sum(similarities) / len(similarities)
euclidean_similarity_df = euclidean_similarity_df.astype(float)

In [16]:
# MinMax-Scaling
scaler = MinMaxScaler()
scaled_tag = euclidean_similarity_df.copy()
scale_col= list(euclidean_similarity_df.columns)

scaled_tag[scale_col] = scaler.fit_transform(scaled_tag[scale_col])
scaled_tag.head()

,1,2,3,4,5,6,7,8,9,10,...,191005,193565,193567,193571,193573,193581,193583,193585,193587,193609
1,0.000000,0.124990,0.107341,0.211414,0.100690,0.104114,0.415405,0.112243,0.246978,0.071301,...,0.174780,0.087411,0.114358,0.085522,0.107854,0.306004,0.261528,0.146815,0.264908,0.164475
2,0.119877,0.000000,0.114716,0.191936,0.096492,0.104695,0.361439,0.078096,0.168087,0.108556,...,0.180921,0.120704,0.122646,0.115784,0.122875,0.298168,0.255583,0.144462,0.259898,0.214101
3,0.100549,0.112041,0.000000,0.130016,0.036301,0.028921,0.432429,0.109526,0.235365,0.069710,...,0.165748,0.072027,0.066779,0.068099,0.064941,0.304008,0.249053,0.074463,0.250940,0.193898
4,0.190499,0.180325,0.125067,0.000000,0.131684,0.129027,0.461052,0.183112,0.275973,0.171904,...,0.253922,0.181797,0.146387,0.169548,0.142070,0.369416,0.323995,0.076886,0.318045,0.263472
5,0.093851,0.093774,0.036121,0.136215,0.000000,0.019885,0.408726,0.084928,0.213230,0.063989,...,0.176391,0.080731,0.048198,0.064583,0.051447,0.312269,0.259206,0.086971,0.262978,0.202140


In [17]:
scaled_tag.to_csv('/content/drive/MyDrive/23 U 4-2/빅데이터처리및응용/1211 팀프로젝트/Final Data/'+ '1210 EucSim_Tag.csv', encoding= 'utf-8', index= False)

### 4.5 Euclidean - Storyline

In [10]:
movie_sample= pd.read_csv(path+ "1210 all_pp.csv")
movie_sample= movie_sample[['movieId', 'storyline']]
movie_sample.head()

,movieId,storyline
0,1,"[0.019509797799400985,-0.022897314243508795,-0..."
1,2,"[0.015569244019747148,-0.0014084530279929327,0..."
2,3,"[0.0034510755504015833,-0.013722939333092654,-..."
3,4,"[0.005609218422950317,-0.022722859384838298,0...."
4,5,"[0.004049535797726687,-0.010654402755038893,-0..."


In [11]:
# Euclidean Similarity
feature_columns = ['storyline']

def list_to_array(lst):
    return np.array(ast.literal_eval(lst)) if isinstance(lst, str) else np.array(lst)

for col in feature_columns:
    movie_sample[col] = movie_sample[col].apply(list_to_array)

euclidean_similarities = {}
for col in feature_columns:
    euclidean_similarities[col] = euclidean_distances(movie_sample[col].tolist())

movie_ids = movie_sample['movieId'].tolist()
euclidean_similarity_df = pd.DataFrame(index=movie_ids, columns=movie_ids)
for i in range(len(movie_ids)):
    for j in range(len(movie_ids)):
        similarities = []
        for col in feature_columns:
            similarities.append(euclidean_similarities[col][i][j])
        euclidean_similarity_df.at[movie_ids[i], movie_ids[j]] = sum(similarities) / len(similarities)
euclidean_similarity_df = euclidean_similarity_df.astype(float)

In [12]:
# MinMax-Scaling
scaler = MinMaxScaler()
scaled_story = euclidean_similarity_df.copy()
scale_col= list(euclidean_similarity_df.columns)

scaled_story[scale_col] = scaler.fit_transform(scaled_story[scale_col])
scaled_story.head()

,1,2,3,4,5,6,7,8,9,10,...,191005,193565,193567,193571,193573,193581,193583,193585,193587,193609
1,0.000000,0.213456,0.178592,0.269044,0.123127,0.177947,0.164903,0.181047,0.265710,0.277534,...,0.267078,0.190246,0.254123,0.230956,0.299433,0.190738,0.322706,0.208583,0.250034,0.163993
2,0.212579,0.000000,0.241955,0.164672,0.187706,0.124325,0.132844,0.179486,0.170515,0.175645,...,0.197143,0.159691,0.361381,0.292095,0.228091,0.131367,0.189538,0.187006,0.174220,0.176395
3,0.182259,0.247942,0.000000,0.295371,0.194079,0.202291,0.189773,0.216725,0.263897,0.320974,...,0.325880,0.275543,0.273592,0.240984,0.317502,0.209289,0.324147,0.276494,0.237606,0.161072
4,0.247463,0.152088,0.266211,0.000000,0.207803,0.165264,0.176752,0.210280,0.219885,0.229430,...,0.225947,0.206633,0.384035,0.299300,0.219385,0.186848,0.179545,0.230391,0.207745,0.209658
5,0.120405,0.184314,0.185970,0.220931,0.000000,0.167535,0.119126,0.171958,0.237998,0.253173,...,0.255053,0.194058,0.299974,0.234561,0.277507,0.179045,0.287468,0.189051,0.234620,0.157032


In [13]:
scaled_story.to_csv('/content/drive/MyDrive/23 U 4-2/빅데이터처리및응용/1211 팀프로젝트/Final Data/'+ '1210 EucSim_Story.csv', encoding= 'utf-8', index= False)